Erkennung von Herzversagen
===
Eine der vielen Möglichkeiten, in welchen personenbezogene Daten verarbeitet werden ist im Gesundheitswesen. So können zum Beispiel Modelle aus dem Maschinellen Lernen eingesetzt werden, um Krebs oder andere Krankheiten zu erkennen, welche aber wiederum persönliche Informationen des Patienten benötigen. Als Anwendungsfall wird in diesem Beispiel die Erkennung von Herzversagen anhand von verschiedenen Merkmalen getestet.<br />
https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction

In [92]:
from heNet import Network, FullyConnectedLayer, ActivationLayer
from heNet import square, square_prime, sigmoid, sigmoid_prime, mse, mse_prime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tenseal as ts

heart_csv = pd.read_csv("../data/heart.csv")

Die rohen Daten können aber nicht direkt zum Training verwendet werden und werden zuerst aufgeteilt, skaliert und transformiert in die korrekte Eingabeform.

In [93]:
heart_dataframe = pd.get_dummies(heart_csv)

y = heart_dataframe["HeartDisease"]
heart_dataframe.drop(["HeartDisease"], axis = 1, inplace = True, errors = "ignore")

X_train, X_test, y_train, y_test = train_test_split(heart_dataframe, y, test_size=0.25) # TODO

scaler = MinMaxScaler() 
X_test_normal = scaler.fit_transform(X_test)

X_train = np.array([[data] for data in scaler.fit_transform(X_train)])
X_test = np.array([[data] for data in scaler.fit_transform(X_test)])
y_train = np.array([[label] for label in y_train.to_numpy()])
y_test =  np.array([[label] for label in y_test.to_numpy()])

Die hier erstellte Architektur besitzt 3 Schichten mit zwei Quadratischen Aktivierungsfunktionen und einer Sigmoid Aktivierungsfunktion am Ende für das Training. Die Dimension der einzelnen Schichten nach der Eingabeschicht wurde niedrig gehalten, um die Größe der verarbeiteten Zahlen zu verringern. Ansonsten würden die Quadratischen Aktivierungsfunktionen zu einem Problem werden.

In [94]:
net = Network(100)
net.add(FullyConnectedLayer(20, 10))
net.add(ActivationLayer(square, square_prime))
net.add(FullyConnectedLayer(10, 1))
net.add(ActivationLayer(sigmoid, sigmoid_prime))

net.use(mse, mse_prime)
net.fit(X_train, y_train, epochs=400, learning_rate=0.001)

epoch 1/400   error=0.241274
epoch 101/400   error=0.098040
epoch 201/400   error=0.094395
epoch 301/400   error=0.091489


Das trainierte Modell wird an dieser Stelle getestet auf einem Testdatensatz und produziert dabei entweder die Ausgabe, dass der Patient ein Risiko für Herzversagen hat oder eben nicht.

In [95]:
out = net.predict(X_test)
correct = 0
for idx,_ in enumerate(out):
    result = 1 if np.squeeze(out[idx]) > 0.5 else 0
    if result == np.squeeze(y_test[idx]):
        correct = correct + 1
print(f"Im Testdatensatz wurden {correct} von {len(out)} richtig klassifiziert --> {round(correct/len(out),3)}")

Im Testdatensatz wurden 196 von 230 richtig klassifiziert --> 0.852


Inferenz auf verschlüsselten Daten
---
Nachdem das HE freundliche Modell erstellt wurde kann dieses nun auch auf den verschlüsselten Daten getestet werden. Verwendet wird dabei der zuvor getestete Teil des Datensatz.

In [96]:
bits_scale = 26
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[31, bits_scale, bits_scale, bits_scale, 31]
)
context.global_scale = pow(2, bits_scale)
context.generate_galois_keys()

e_X_test = np.array([[ts.ckks_vector(context, [value]) for value in data] for data in X_test_normal])

Als nächstes wird die letzte Schicht im Modell entfernt, da die verschlüsselten Daten nicht mit der normalen Sigmoid Funktion verrechnet werden können.

In [97]:
net.layers = net.layers[:-1]

Der letzte und längste Schritt ist das eigentliche Testen der verschlüsselten Daten.

In [98]:
e_out = net.predict(e_X_test)
correct = 0
d_out = []
for idx,_ in enumerate(e_out):
    result = 1 if sigmoid(e_out[idx][0][0].decrypt()[0]) > 0.5 else 0
    d_out.append(result)
    if result == np.squeeze(y_test[idx]):
        correct = correct + 1
print(f"Im Testdatensatz wurden {correct} von {len(e_out)} richtig klassifiziert --> {round(correct/len(e_out),3)}")

Im Testdatensatz wurden 196 von 230 richtig klassifiziert --> 0.852


* Positiv: Genauigkeit zwischen unverschlüsselte und verschlüsselte Test sind gleich
* Negativ: Laufzeit hängt stark von der Architektur und Größe der verschlüsselten Objekte ab

Zum Vergleich wird noch überprüft ob jede Eingabe auch die gleiche Ausgabe produziert hat.

In [99]:
u_out = [1 if np.squeeze(value) > 0.5 else 0 for value in out]
print(u_out == d_out)

True
